In [16]:
import pandas as pd
import itertools
id = [1, 2, 3, 4, 5, 6]
origin = ['SFO', 'SFO', 'SFO', 'DFW', 'DFW', 'JFK']
destination = ['JFK', 'DFW', 'MCO', 'MCO', 'JFK', 'LHR']
cost = [500, 200, 400, 100, 200, 1000]
da_flights=pd.DataFrame({'id':id, 'origin':origin, 'destination':destination, 'cost':cost})
import pandas as pd
import itertools

df = pd.DataFrame(list(itertools.product(da_flights['origin'].unique(), da_flights['destination'].unique())),
                  columns=['origin', 'destination'])
df = df[df['origin'] != df['destination']]

connections_1 = pd.merge(da_flights, da_flights,
                         how='left', left_on='destination', right_on='origin', suffixes=['_0', '_1'])
connections_2 = pd.merge(connections_1, da_flights[['origin', 'destination', 'cost']],
                         how='left', left_on='destination_1', right_on='origin', suffixes=['', '_2']).fillna(0)
connections_2.columns = ['id_0', 'origin_0', 'destination_0', 'cost_0', 'id_1', 'origin_1', 'destination_1',
                         'cost_1', 'origin_2', 'destination_2', 'cost_2']
connections_2['cost_v1'] = connections_2['cost_0'] + connections_2['cost_1']
connections_2['cost_v2'] = connections_2['cost_0'] + connections_2['cost_1'] + connections_2['cost_2']

result = pd.merge(df, da_flights[['origin', 'destination', 'cost']], how='left', on=['origin', 'destination'])

result = pd.merge(result, connections_2[['origin_0', 'destination_1', 'cost_v1']], how='left',
                  left_on=['origin', 'destination'], right_on=['origin_0', 'destination_1'])

result = pd.merge(result, connections_2[['origin_0', 'destination_2', 'cost_v2']], how='left',
                  left_on=['origin', 'destination'], right_on=['origin_0', 'destination_2'])
result['min_price'] = result[['cost', 'cost_v1', 'cost_v2']].min(axis=1)
result[~result['min_price'].isna()][['origin', 'destination', 'min_price']]


,origin,destination,min_price
0,SFO,JFK,400.0
1,SFO,DFW,200.0
2,SFO,MCO,300.0
3,SFO,LHR,1400.0
4,DFW,JFK,200.0
5,DFW,MCO,100.0
6,DFW,LHR,1200.0
9,JFK,LHR,1000.0
